In [1]:
!pip install fastai==0.7.0 --no-deps

In [2]:
!pip install torch==0.1.2

  Using cached https://files.pythonhosted.org/packages/c0/66/2a0696bdac7ed0a99b65f6d8a5da7bc2583835ea04cac058bcd982cbb9ef/torch-0.1.2.tar.gz


    ERROR: Command errored out with exit status 1:
     command: 'c:\users\82103\venv\py367july\scripts\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\82103\\AppData\\Local\\Temp\\pip-install-sj64gaje\\torch\\setup.py'"'"'; __file__='"'"'C:\\Users\\82103\\AppData\\Local\\Temp\\pip-install-sj64gaje\\torch\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base pip-egg-info
         cwd: C:\Users\82103\AppData\Local\Temp\pip-install-sj64gaje\torch\
    Complete output (5 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\82103\AppData\Local\Temp\pip-install-sj64gaje\torch\setup.py", line 12, in <module>
        from tools.setup_helpers.env import check_env_flag
    ModuleNotFoundError: No module named 'tools.setup_helpers'
    --------------------------------

In [3]:
!pip install bcolz
!pip install isoweek
!pip install pandas_summary

In [4]:
import fastai
from fastai.conv_learner import *
from cv2 import resize
%matplotlib inline

In [5]:
class SaveFeatures():
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.features = torch.tensor(output,requires_grad=True).cuda()
    def close(self):
        self.hook.remove()

In [6]:
class FilterVisualizer():
    def __init__(self, size=56, upscaling_steps=12, upscaling_factor=1.2):
        self.size, self.upscaling_steps, self.upscaling_factor = size, upscaling_steps, upscaling_factor
        self.model = vgg16(pre=True).cuda().eval()
        set_trainable(self.model, False)

    def visualize(self, layer, filter, lr=0.1, opt_steps=20, blur=None):
        sz = self.size
        img = np.uint8(np.random.uniform(150, 180, (sz, sz, 3)))/255  # generate random image
        activations = SaveFeatures(list(self.model.children())[layer])  # register hook

        for _ in range(self.upscaling_steps):  # scale the image up upscaling_steps times
            train_tfms, val_tfms = tfms_from_model(vgg16, sz)
            img_var = V(val_tfms(img)[None], requires_grad=True)  # convert image to Variable that requires grad
            optimizer = torch.optim.Adam([img_var], lr=lr, weight_decay=1e-6)
            for n in range(opt_steps):  # optimize pixel values for opt_steps times
                optimizer.zero_grad()
                self.model(img_var)
                loss = -activations.features[0, filter].mean()
                loss.backward()
                optimizer.step()
            img = val_tfms.denorm(img_var.data.cpu().numpy()[0].transpose(1,2,0))
            self.output = img
            sz = int(self.upscaling_factor * sz)  # calculate new image size
            img = cv2.resize(img, (sz, sz), interpolation = cv2.INTER_CUBIC)  # scale image up
            if blur is not None: img = cv2.blur(img,(blur,blur))  # blur image to reduce high frequency patterns
        self.save(layer, filter)
        activations.close()
        
    def save(self, layer, filter):
        plt.imsave("layer_"+str(layer)+"_filter_"+str(filter)+".jpg", np.clip(self.output, 0, 1))

In [7]:
layer = 40
filter = 265

FV = FilterVisualizer(size=56, upscaling_steps=12, upscaling_factor=1.2)
FV.visualize(layer, filter, blur=5)

img = PIL.Image.open("layer_"+str(layer)+"_filter_"+str(filter)+".jpg")
plt.figure(figsize=(7,7))
plt.imshow(img)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to C:\Users\82103/.cache\torch\checkpoints\vgg16_bn-6c64b313.pth
100%|███████████████████████████████████████████████████████████████████████████████| 528M/528M [00:17<00:00, 32.3MB/s]


AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [8]:
import torch
print(torch.__version__)

1.2.0+cu92


In [9]:
import pandas
print(pandas.__version__)

0.25.0
